In [ ]:
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
import seaborn as sns

import re
from fake_useragent import UserAgent
import requests 
from bs4 import BeautifulSoup
from pymongo import MongoClient
import chromedriver_binary
from time import sleep
from selenium import webdriver
from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.select import Select
from selenium.webdriver.support.expected_conditions import NoAlertPresentException
from selenium.webdriver.support.expected_conditions import NoSuchElementException
from selenium.webdriver.support.expected_conditions import NoSuchFrameException
from selenium.webdriver.support.expected_conditions import StaleElementReferenceException
from selenium.webdriver.support.expected_conditions import WebDriverException 

In [ ]:
# URL STRUCTURE :

# url = "https://economictimes.indiatimes.com/archive/year-" + str(year_name) + "," + "month-" + str(month_no) + ".cms"

><h1> SELENIUM VERISON - (COMPLETE SELENIUM) :<h1>

In [ ]:
# for each_year in years_range:
    
#     dict_record = {}
    
#     # find the total no of days in that
    
#     for each_month_no in months_range:
        
#         # frame url for this month of this year
#         year_month_url = "https://economictimes.indiatimes.com/archive/year-" + str(each_year) + "," + "month-" + str(each_month_no) + ".cms"
#         print(year_month_url)
        
#         dict_record['year_month_url'] = year_month_url
#         dict_record['year'] = each_year
#         dict_record['month'] = each_month_no
        
#         # open the url for finding links of all dates present on this url
#         responseFromYearMonthUrl = requests.get(year_month_url)
#         soup = BeautifulSoup(responseFromYearMonthUrl.text, 'lxml')
        
#         # FIND ALL THE DATES LINKS TO ARCHIVES
        
#         all_dates_links_for_this_month_list = []
#         all_dates_links_for_this_month = driver.find_elements_by_xpath(r'//table[@id="calender"]//tr//a[contains(@href,"/archivelist/year-") and contains(@href,"month-") and contains(@href, "starttime")]')
        
#         for each_date_link in all_dates_links_for_this_month:
#             all_dates_links_for_this_month_list.append(each_date_link.get_attribute('href'))
        
#         all_dates_links_for_this_month_list = set(all_dates_links_for_this_month_list)
        
#         for each_date_link in all_dates_links_for_this_month_list:
            
#             # go to that date link and it now displays all the archives for the day 
#             driver.get(each_date_link)
            
#             dict_temp['timestamp'] = driver.find_element_by_xpath(r'//*[@id="pageContent"]/span/table[1]/tbody/tr[2]/td/b').text
            
#             all_articles_links_for_the_day_list = [] # all articles link will be ready in this list
#             all_articles_links_for_the_day = driver.find_elements(By.XPATH, "//td[@class='contentbox5']//ul//li//a")
            
#             for each_article_link in all_articles_links_for_the_day:
#                 all_articles_links_for_the_day_list.append(each_article_link.get_attribute('href'))
            
#             if len(all_articles_links_for_the_day_list) > 0:
                
#                 for each_article_link in all_articles_links_for_the_day_list:
                    
#                     # open this article url to see the article
#                     driver.get(each_article_link)
#                     sleep(5)

#                     # Finally on the article page
#                     # if 'Industry›Energy›Power'  or 'Industry›Energy›Oil & Gas' then scrape else move to next article webpage :

#                     industry_check = driver.find_element_by_xpath(r"//div[@class='clr breadCrumb']//span[2]").text.strip()
#                     energy_check = driver.find_element_by_xpath(r"//div[@class='clr breadCrumb']//span[3]").text.strip()
#                     power_check = driver.find_element_by_xpath(r"//div[@class='clr breadCrumb']//span[4]").text.strip()
#                     oilAndGas_check =driver.find_element_by_xpath(r"//div[@class='clr breadCrumb']//span[4]").text.strip()

#                     if ((industry_check=='Industry' and energy_check=='Energy') and (power_check=='Power' or oilAndGas_check=='Oil & Gas')):
#                         # scrape report
#                         articleHeadline = driver.find_element(By.XPATH, "//h1[@class='clearfix title']").text.strip()
#                         article_text = driver.find_element(By.XPATH, "//div[@class='article_block gaDone'][1]//article[contains(@data-artidate,'')]//div[@class='artText']").text
#                         dict_temp['article heading'] = articleHeadline
#                         dict_temp['article_text'] = article_text
#                         print("FOUND -----------")
#                         print(dict_temp)
#                         break


#                     else:
#                         # no evidence for energy>power or energy>oil & gas
#                         print('NOT PRESENT------')
#                         continue

><h1> BEAUTIFULSOUP VERISON - (WHEREVER POSSIBLE) :<h1>

## Mongo Collection - I  (for articles storage - MAIN COLLECTION):

In [ ]:
client = MongoClient()
db_EconomicTimes = client['db_EconomicTimes']
collection_articles = db_EconomicTimes['collection_articles']

## Mongo Collection- II (for keeping pointer/track of last day worked on):

In [ ]:
client = MongoClient()
db_EconomicTimes = client['db_EconomicTimes']
collection_last_day_flag = db_EconomicTimes['collection_last_day_flag']

## Mongo Collection- II (for keeping pointer/track of last month covered):

In [ ]:
client = MongoClient()
db_EconomicTimes = client['db_EconomicTimes']
collection_last_day_month_flag = db_EconomicTimes['collection_last_day_month_flag']

In [ ]:
def getAllDatesLinks(page_source):
    
    
    '''
    On the page that shows the date links of that month inside a calender format, 
    find and store all the days links
    '''
    soup = BeautifulSoup(page_source, 'lxml')
    all_dates_links_for_this_month_list = []
#     soup = BeautifulSoup(driver.page_source ,'lxml')
    
    # Target Table - calender
    targetTable = soup.find('table', attrs={'id':'calender'})
    
#     return targetTable 
    sleep(2)
    # all tds under the table - calender
    tableRowDatas = targetTable.findAll('td')

    for each in tableRowDatas:
        
        try:
            date_link_a_tag = each.find('a',attrs={'href':re.compile(r"/archivelist/year-"), 
                                            'href':re.compile(r"month-"), 
                                            'href':re.compile(r"starttime")})


            # If this is a date link then , as text, it should contain date no Ex <a tag>12</a tag>
            if int(date_link_a_tag.text.strip()) in list(np.arange(1, 32)):  
                all_dates_links_for_this_month_list.append('https://economictimes.indiatimes.com' + date_link_a_tag['href'])
                
        except:
            pass

    return list(set(all_dates_links_for_this_month_list))

In [ ]:
def getALLArchivesLinks(soup):
    
    '''
    After opening a certain date link, we move to page displaying all archives of that day,,
    hence we store all the link for that archives
    
    '''
    
    all_articles_links_for_the_day_list = []

    all_ul_tags = soup.findAll('ul', attrs={'class':'content'})  # max 2 uls
    
    for each_ul_tag in all_ul_tags:
        
        all_a_tags = each_ul_tag.findAll('a')
        
        for each_a_tag in all_a_tags:
        
            all_articles_links_for_the_day_list.append('https://economictimes.indiatimes.com' + each_a_tag['href'])
    
    return list(set(all_articles_links_for_the_day_list))

In [ ]:
# def checkLastTimeStamp():
    
#     each_element_time = {}
    
#     for each_element_time in collection_last_day_flag.find():
#         continue
    
#     if len(each_element_time.keys()) == 0:
#         return None
    
#     last_Latest_Timestamp_covered = each_element_time['last_Timestamp']
    
#     markerTimeStamp = pd.Timestamp(last_Latest_Timestamp_covered) 
#     return markerTimeStamp

In [ ]:
# OPEN THE DRIVER INCOGNITO AND MAXIMIZE :

chrome_options = webdriver.ChromeOptions()
chrome_options.add_argument('--incognito')

driver = webdriver.Chrome(chrome_options=chrome_options, executable_path=r'C:\Users\shubhendra\Downloads\chromedriver_win32\chromedriver.exe')
driver.maximize_window()

In [ ]:
latest_year = pd.Timestamp('today').year
latest_month = pd.Timestamp('today').month

In [ ]:
latest_month

In [ ]:
latest_year

In [ ]:
# collection_last_day_month_flag.delete_one({'year_month_url':'https://economictimes.indiatimes.com/archive/year-'+str(latest_year)+ ',month-'+ str(latest_month)+'.cms'})

In [ ]:
# markerTimeStamp = checkLastTimeStamp()
    
# if markerTimeStamp == None:
#     print("RAW FRESH START")
#     years_range = np.arange(2019, 2000, -1)
#     months_range = np.arange(12, 0, -1)

# else:
#     print("STARTING FROM YEAR : ", markerTimeStamp.year,)
#     print("STARTING FROM MONTH : ", markerTimeStamp.month,)
#     years_range = np.arange(markerTimeStamp.year,  2000, -1)
#     months_range = np.arange(markerTimeStamp.month, 0, -1)


latest_year = pd.Timestamp('today').year
latest_month = pd.Timestamp('today').month

collection_last_day_month_flag.delete_one({'year_month_url':'https://economictimes.indiatimes.com/archive/year-'+str(latest_year)+ ',month-'+ str(latest_month)+'.cms'})


years_range = np.arange(pd.Timestamp('today').year,  2000, -1) 
months_range = np.arange(12, 0 , -1) 

for each_year in years_range:
    
    for each_month_no in months_range:
        
        # Frame url for this month of this year
        year_month_url = "https://economictimes.indiatimes.com/archive/year-" + str(each_year) + "," + "month-" + str(each_month_no) + ".cms"
        
                
        if collection_last_day_month_flag.find_one({'year_month_url':str(year_month_url)}):
            # MONTH ALREADY COVERED, HENCE MOVE TO NEXT MONTH
            print('MONTH ALREADY COVERED')
            continue
        

        ua = UserAgent()
        header = {'User-Agent':ua.chrome}
        responseYearMonthURL = requests.get(year_month_url, headers=header)
        if responseYearMonthURL.status_code >=200 and responseYearMonthURL.status_code <=220:
            
            print("YEAR-MONTH URL EXISTS :-----------> ", year_month_url)

            # OPENED the url for finding links of all dates present on this url inside calender 

            # Find all the day links for this month
            # SELENIUM PAGESOURCE USED :
            driver.get(year_month_url)
            all_dates_links_for_this_month_list = getAllDatesLinks(driver.page_source)
            
            if len(all_dates_links_for_this_month_list) == 0 :
                # IF THE MONTH CONSISTS OF NO DAYS I.E. NO CALENDER DISPLAYED
                # IT MEANS WE ARE IN FUTURE MONTH 
                continue

            for each_date_link in all_dates_links_for_this_month_list:

                if collection_last_day_flag.find_one({'last_flag_day_link_to_AllArchives':each_date_link}):
                    print('DAY ALREADY COVERED')
                    # ALREADY COVERED ALL ACRHIVES FOR THIS DAY
                    continue

                else:
                    # we need to cover Archives for this day
                    # Go to that date link and it now displays all the archives for the day 
                    # SELENIUM USED:
                    sleep(3)
                    print("DAY LINK TO ALL ARCHIVES OF THE DAY : ----------> ", each_date_link)
                    
                    driver.get(each_date_link)
                    sleep(4)
                    driver.find_element_by_tag_name('body').send_keys(Keys.END)
                    sleep(2)
                    driver.find_element_by_tag_name('body').send_keys(Keys.HOME)
                    sleep(4)
                    soup = BeautifulSoup(driver.page_source, 'lxml')
    
                    all_articles_links_for_the_day_list = getALLArchivesLinks(soup) # All articles link will be ready in this list
                    

                    # current date time year value as timestamp from all archives link page
                    timestamp_day_month_year = pd.Timestamp(soup.find('td', attrs={'class':'contentbox5'}).findAll('b')[1].text)
                    
                    # CHECK WHETHER THERE ARE ANY 
                    if len(all_articles_links_for_the_day_list) > 0:

                        for each_article_link in all_articles_links_for_the_day_list:
                            
                            if collection_articles.find_one({'article_url':str(each_article_link)}):
                                # ARTICLE ALREADY SCRAPED AND PRESENT IN COLLECTION.ARTICLES
                                # MOVE TO NEXT ARTICLE LINK
                                continue

                            if len(re.findall(r"/industry/energy/", str(each_article_link), re.IGNORECASE)) > 0:
                                # when the article url mentions '/industry/energy/', then only hit request and scrape
                                # else continue to next article url
                                
                                count_to_break_final_if_large_hitRequest_failure_occurs = 0
                                
                                while True: 
                                    # keep hitting requests till the article link response with code http 200 
                                    sleep(5)
                                    
                                    try:
                                        header = {'User-Agent':ua.chrome}
                                        responseArticlePage = requests.get(each_article_link, headers=header, timeout=10)
                                        
                                        if responseArticlePage.status_code == 200 :
                                            soup = BeautifulSoup(responseArticlePage.text, 'lxml')
                                            break
                                            
                                        if responseArticlePage.status_code >= 400 : 
                                            # IF ARTICLE LINK LEADS TO 404 ERROR OR SOME OTHER ERROR TYPE, THEN BREAK
                                            soup = BeautifulSoup(responseArticlePage.text, 'lxml')
                                            break
                                            
                                    except:
                                        print('Hitting request again to article page ')
                                        count_to_break_final_if_large_hitRequest_failure_occurs += 1
                                        
                                        # WE WONT HIT REQUEST, MORE THAN 10 TIMES TO THE ARTICLE URL 
                                        if count_to_break_final_if_large_hitRequest_failure_occurs >= 10:
                                            print('LIMIT TO HIT REQUEST EXCEEDED MORE THAN 10 TIMES')
                                            break
                                            
                                        else:
                                            continue
                                
                                if count_to_break_final_if_large_hitRequest_failure_occurs >= 10:
                                    print('MOVING TO NEXT ARTICLE URL AS HITTING >=10, FAILED')
                                    continue
                                    
                                
                                try:
                                    breadCrumbDivText = soup.find('div', attrs={'class':'breadCrumb'}).text 
                                except:
                                    # ERROR 404 ON ARTICLE PAGE
                                    print('ERROR 404 OCCURED, MOVING TO NEXT ARTICLE LINK')
                                    continue
                                    
                                if len(re.findall(r"Industry", breadCrumbDivText) ) > 0:
                                    if len(re.findall(r"Energy", breadCrumbDivText) ) > 0:
                                        if len(re.findall(r"Power|Oil & Gas", breadCrumbDivText)) > 0:
                                            
                                            domain = None
                                            if breadCrumbDivText.__contains__('Power'):
                                                domain = 'Power'
                                            
                                            else:
                                                domain = 'Oil & Gas'
                                            
                                            dict_record = {}
                                            
                                            # SCRAPE REPORT :
                                            
                                            try:
                                                articleHeadline = soup.find('h1', attrs={'class':'clearfix title'}).text.strip()

                                                # ARTICLE TEXT :
                                                div_tag_article = soup.find('div', attrs={'class':'Normal'})
                                                article_text = div_tag_article.text.strip()

                                                dict_record['domain'] = domain
                                                dict_record['article_heading'] = articleHeadline
                                                dict_record['article_text'] = article_text
                                                dict_record['article_url'] = each_article_link
                                                dict_record['year_name'] = str(each_year)
                                                dict_record['month_name'] = str(each_month_no)
                                                dict_record['day_name'] = str(pd.Timestamp(timestamp_day_month_year).day)
                                                dict_record['day_link_to_allArchives'] = each_date_link
                                                dict_record['year_month_url'] = year_month_url
                                                dict_record['timestamp'] = str(timestamp_day_month_year)
                                            
                                            except:
                                                # SOME ERROR OCCURED WHILE FINDING THE DATA CONTENT ON THE ARTICLE URL PAGE, HENCE IGNORING THAT ARTICLE
                                                # MOVE TO NEXT ARTICLE URL 
                                                print('ERROR ON FINDING DATA')
                                                continue

                                            print("FOUND/INSERTING ARTICLE ON - INDUSTRY/ENERGY :")
                                            collection_articles.insert_one(dict_record)
#                                             print(dict_record)
                        
                        print("<------------------COVERED ALL THE ARTICLES FOR THE DAY-------------->")
                        
                        # WHEN ALL ARTICLES FOR THE DAY ARE COVERED, THEN INSERT INTO FLAG COLLECTION :
                        collection_last_day_flag.insert_one({'last_flag_day_link_to_AllArchives':str(each_date_link), 
                                                         'year_name':str(each_year), 
                                                         'month_name':str(each_month_no),
                                                         'last_Timestamp':str(timestamp_day_month_year),
                                                         'year_month_url':str(year_month_url)   
                                                        })
                    
            # WHEN ALL DAYS FOR THE MONTH ARE NOT COVERED, THEN INSERT INTO MONTH FLAG COLLECTION :
            total_days_of_this_month = pd.Timestamp(str(each_month_no) + ' ' + str(each_year)).days_in_month
            if total_days_of_this_month != len(all_dates_links_for_this_month_list):
                print("We are in the current latest month of the latest year")
                # if total no of days of this month of this year is not equal to the total days links found for this month for this year
                # then dont insert this month into month_flag_collection
                continue
            
            
            # WHEN ALL DAYS FOR THE MONTH ARE COVERED, THEN INSERT INTO MONTH FLAG COLLECTION :
            # I.E WHEN ALL THE ARCHIVES FROM ALL THE DAYS FOR THE MONTH HAVE BEEN SCRAPED 
            collection_last_day_month_flag.insert_one({'year_month_url':str(year_month_url)})

In [ ]:
year_month_url

In [ ]:
each_article_link

In [ ]:
s = requests.get(each_article_link)
soup = BeautifulSoup(s.text, 'lxml').find('div', attrs={'class':'Normal'})
soup

# TEST CODE FOR A DAY - ALL ARCHIVES  :

In [ ]:
# all_dates_links_for_this_month_list = getALLArchivesLinks(BeautifulSoup(driver.page_source, 'lxml'))

# for each_date_link in all_dates_links_for_this_month_list:
#     print("current_date link : ", each_date_link)
    
#     # Go to that date link and it now displays all the archives for the day 
#     # BEAUTIFULSOUP USED:
#     responseAllArchivesForDayPage = requests.get('https://economictimes.indiatimes.com/'+ str(each_date_link), headers={'User-Agent':'Chrome'}, timeout=5)
#     soup = BeautifulSoup(responseAllArchivesForDayPage.text, 'lxml')
#     all_articles_links_for_the_day_list = getALLArchivesLinks(soup) # All articles link will be ready in this list


#     if len(all_articles_links_for_the_day_list) > 0:
#         print("No of total arrticles for the day :",len(all_articles_links_for_the_day_list))
        
#         for each_article_link in all_articles_links_for_the_day_list:
            
#             if len(re.findall(r"/Industry/Energy/", str(each_article_link), re.IGNORECASE)) > 0:
#                 # when the article url mentions 'enery' 'industry' ,then only hit requesta and scrape
#                 # else continue to next article url
            
#             print("Current article url :", each_article_link)
#             # Open this article url to see the article
#             # BEAUTIFULSOUP
#             sleep(2)
#             responseArticlePage = requests.get('https://economictimes.indiatimes.com/'+str(each_article_link), headers={'User-Agent':'Chrome'})
#             soup = BeautifulSoup(responseArticlePage.text, 'lxml')

#             # Finally on the article page
#             # if 'Industry›Energy›Power'  or 'Industry›Energy›Oil & Gas' then scrape else move to next article webpage :
            
#             breadCrumbDivText = soup.find('div', attrs={'class':'breadCrumb'}).text 
            
#             if len(re.findall(r"Industry", breadCrumbDivText) ) > 0:
#                 if len(re.findall(r"Energy", breadCrumbDivText) ) > 0:
#                     if len(re.findall(r"Power|Oil & Gas", breadCrumbDivText)) > 0:

#                         # SCRAPE REPORT :
#                         articleHeadline = soup.find('h1', attrs={'class':'clearfix title'}).text.strip()

#                         # ARTICLE TEXT :
#                         div_tag_article = soup.find('div', attrs={'class':'Normal'})
#                         article_text = div_tag_article.text.strip()

#                         print(articleHeadline, "    ",  article_text)

#                         print("FOUND -----------")
# #                         print(dict_temp)